<a href="https://colab.research.google.com/github/Tamiyo22/AppointmentSchedulingApplication/blob/master/recent_earthquakes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install folium pandas requests


In [12]:
import requests
import pandas as pd

url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_day.geojson'
response = requests.get(url)
data = response.json()

# Normalize the JSON data to a pandas DataFrame
earthquakes = pd.json_normalize(data['features'])


In [13]:
# Extract relevant information
earthquakes['place'] = earthquakes['properties.place']
earthquakes['magnitude'] = earthquakes['properties.mag']
earthquakes['time'] = pd.to_datetime(earthquakes['properties.time'], unit='ms')
earthquakes['longitude'] = earthquakes['geometry.coordinates'].apply(lambda x: x[0])
earthquakes['latitude'] = earthquakes['geometry.coordinates'].apply(lambda x: x[1])


In [14]:
import folium

# Initialize a map centered around an average location
map_center = [earthquakes['latitude'].mean(), earthquakes['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=2)

# Add earthquake locations to the map
for idx, row in earthquakes.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['magnitude'],
        popup=f"{row['place']}, Mag: {row['magnitude']}",
        color='red',
        fill=True,
        fill_color='red'
    ).add_to(m)

# Save the map to an HTML file
m.save('earthquakes_map.html')



In [15]:
earthquakes.head()

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates,place,magnitude,time,longitude,latitude
0,Feature,us7000ms1a,4.50,"5 km SSE of Coquimbo, Chile",1718231218295,1718232106040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,mb,earthquake,"M 4.5 - 5 km SSE of Coquimbo, Chile",Point,"[-71.3121, -29.9937, 49.219]","5 km SSE of Coquimbo, Chile",4.50,2024-06-12 22:26:58.295,-71.312100,-29.9937
1,Feature,us7000ms15,4.30,"164 km NNW of Hicks Bay, New Zealand",1718229013822,1718230196040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,mb,earthquake,"M 4.3 - 164 km NNW of Hicks Bay, New Zealand",Point,"[177.8283, -36.1646, 249.648]","164 km NNW of Hicks Bay, New Zealand",4.30,2024-06-12 21:50:13.822,177.828300,-36.1646
2,Feature,us7000ms0i,4.30,"24 km NE of Orcopampa, Peru",1718223501688,1718225325040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,mb,earthquake,"M 4.3 - 24 km NE of Orcopampa, Peru",Point,"[-72.1974, -15.1004, 117.658]","24 km NE of Orcopampa, Peru",4.30,2024-06-12 20:18:21.688,-72.197400,-15.1004
3,Feature,ak0247jjrmlu,2.80,"76 km SW of Nikolski, Alaska",1718222034311,1718231156178,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,ml,earthquake,"M 2.8 - 76 km SW of Nikolski, Alaska",Point,"[-169.7353, 52.5068, 26.5]","76 km SW of Nikolski, Alaska",2.80,2024-06-12 19:53:54.311,-169.735300,52.5068
4,Feature,pr71452438,2.86,"37 km NNE of Cruz Bay, U.S. Virgin Islands",1718221033260,1718222325220,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,md,earthquake,"M 2.9 - 37 km NNE of Cruz Bay, U.S. Virgin Isl...",Point,"[-64.6093333333333, 18.6215, 2.3]","37 km NNE of Cruz Bay, U.S. Virgin Islands",2.86,2024-06-12 19:37:13.260,-64.609333,18.6215
